# PPR Dependence

I'm interested in how dramatically the value-based rankings of players change with the points per reception. My guess is that there will be large swings between 0, 0.5 and 1.0 points per reception. The implication would be that one-size fits all rankings are BS.

In [136]:
import os
import pandas as pd

data = pd.read_csv(os.path.join("..", "..", "data", "processed", "Projections-2019.csv"))

data.head()

,key,name,pos,team,bye,adp,kick_20_29,kick_extra_points,kick_40_49,reception_yds,...,pass_yds,reception_tds,pass_ints,df_sacks,df_tds,rush_yds,kick_30_39,two_pts,df_ints,kick_50
0,barkley_RB_NYG,Saquon Barkley,RB,NYG,11.0,1.3,NaN,NaN,NaN,666.6,...,NaN,3.2,0.0,NaN,NaN,1267.0,NaN,0.6,NaN,NaN
1,elliott_RB_DAL,Ezekiel Elliott,RB,DAL,8.0,2.3,NaN,NaN,NaN,552.3,...,NaN,2.8,0.0,NaN,NaN,1364.0,NaN,NaN,NaN,NaN
2,mccaffrey_RB_CAR,Christian McCaffrey,RB,CAR,7.0,3.0,NaN,NaN,NaN,799.9,...,NaN,4.7,0.0,NaN,NaN,1055.6,NaN,0.0,NaN,NaN
3,kamara_RB_NO,Alvin Kamara,RB,NO,9.0,3.5,NaN,NaN,NaN,748.8,...,NaN,4.1,0.0,NaN,0.1,943.0,NaN,1.6,NaN,NaN
4,gordon_RB_LAC,Melvin Gordon,RB,LAC,12.0,5.5,NaN,NaN,NaN,485.4,...,NaN,3.0,0.0,NaN,NaN,1005.8,NaN,0.5,NaN,NaN


In [137]:
# create projections for each player

import numpy as np

points = {  # standard ESPN scoring
    "pass_yds": 0.04,
    "pass_tds": 4.0,
    "pass_ints": -2.0,
    "receptions": 0.0,  # the stat that matters here
    "reception_yds": 0.1,
    "reception_tds": 6.0,
    "rush_yds": 0.1,
    "rush_tds": 6.0,
    "fumbles": -2.0,
    "two_pts": 2.0,
    "kick_extra_points": 1.0,
    "kick_0_19": 3.0,
    "kick_20_29": 3.0,
    "kick_30_39": 3.0,
    "kick_40_49": 4.0,
    "kick_50": 5.0,
    "df_ints": 2.0,
    "df_tds": 6.0,
    "df_sacks": 1.0,
    "df_points_allowed_per_game": 0,
    "df_fumbles": 2.0,
    "df_safeties": 2.0,
}

data = data.fillna(0)

for ppr in [0.0, 0.5, 1.0]:
    points["receptions"] = ppr

    proj = "proj-" + str(ppr)  # col name
    data[proj] = data.apply(lambda x: sum(x[col] * pts for col, pts in points.items() if x[col]), axis=1)

data.head()

,key,name,pos,team,bye,adp,kick_20_29,kick_extra_points,kick_40_49,reception_yds,...,df_sacks,df_tds,rush_yds,kick_30_39,two_pts,df_ints,kick_50,proj-0.0,proj-0.5,proj-1.0
0,barkley_RB_NYG,Saquon Barkley,RB,NYG,11.0,1.3,0.0,0.0,0.0,666.6,...,0.0,0.0,1267.0,0.0,0.6,0.0,0.0,263.36,304.61,345.86
1,elliott_RB_DAL,Ezekiel Elliott,RB,DAL,8.0,2.3,0.0,0.0,0.0,552.3,...,0.0,0.0,1364.0,0.0,0.0,0.0,0.0,255.23,290.53,325.83
2,mccaffrey_RB_CAR,Christian McCaffrey,RB,CAR,7.0,3.0,0.0,0.0,0.0,799.9,...,0.0,0.0,1055.6,0.0,0.0,0.0,0.0,249.75,296.45,343.15
3,kamara_RB_NO,Alvin Kamara,RB,NO,9.0,3.5,0.0,0.0,0.0,748.8,...,0.0,0.1,943.0,0.0,1.6,0.0,0.0,247.98,292.18,336.38
4,gordon_RB_LAC,Melvin Gordon,RB,LAC,12.0,5.5,0.0,0.0,0.0,485.4,...,0.0,0.0,1005.8,0.0,0.5,0.0,0.0,219.12,245.67,272.22


In [138]:
# add value to each player

positions = ["QB", "RB", "WR", "TE", "DST", "K"]

data = data[data.adp > 0]

for pts in ["proj-0.0", "proj-0.5", "proj-1.0"]:
    # map from each position to the numberof players in that position
    # drafted during the first 100 picks of a draft
    # that's the index of the replacement player
    # see: https://www.footballguys.com/05vbdrevisited.htm
    replacement = {}
    for pos in positions:
        replacement[pos] = len(data[(data.pos == pos) & (data.adp <= 100.0)])

    replace_pts = {}
    for pos, ind in replacement.items():
        replace_pts[pos] = data[data.pos == pos][ind:ind+1][pts].values[0]
                                                              
    value = pts + "-value"
    data[value] = data.apply(lambda x: x[pts] - replace_pts[x["pos"]], axis=1)
    
data.head()

,key,name,pos,team,bye,adp,kick_20_29,kick_extra_points,kick_40_49,reception_yds,...,kick_30_39,two_pts,df_ints,kick_50,proj-0.0,proj-0.5,proj-1.0,proj-0.0-value,proj-0.5-value,proj-1.0-value
0,barkley_RB_NYG,Saquon Barkley,RB,NYG,11.0,1.3,0.0,0.0,0.0,666.6,...,0.0,0.6,0.0,0.0,263.36,304.61,345.86,174.35,205.55,236.75
1,elliott_RB_DAL,Ezekiel Elliott,RB,DAL,8.0,2.3,0.0,0.0,0.0,552.3,...,0.0,0.0,0.0,0.0,255.23,290.53,325.83,166.22,191.47,216.72
2,mccaffrey_RB_CAR,Christian McCaffrey,RB,CAR,7.0,3.0,0.0,0.0,0.0,799.9,...,0.0,0.0,0.0,0.0,249.75,296.45,343.15,160.74,197.39,234.04
3,kamara_RB_NO,Alvin Kamara,RB,NO,9.0,3.5,0.0,0.0,0.0,748.8,...,0.0,1.6,0.0,0.0,247.98,292.18,336.38,158.97,193.12,227.27
4,gordon_RB_LAC,Melvin Gordon,RB,LAC,12.0,5.5,0.0,0.0,0.0,485.4,...,0.0,0.5,0.0,0.0,219.12,245.67,272.22,130.11,146.61,163.11


In [152]:
# add ranks, sort and print top 50 players

from sklearn.metrics import r2_score
import matplotlib.pyplot as plt

ranks = ["0.0-rank", "0.5-rank", "1.0-rank"]

for i, pts in enumerate(["proj-0.0-value", "proj-0.5-value", "proj-1.0-value"]):
    data = data.sort_values(pts, ascending=False)
    data[ranks[i]] = np.arange(len(data)) + 1
    
data["diff"] = data[ranks[2]] - data[ranks[0]]
data["abs_diff"] = abs(data["diff"])
data = data.sort_values(ranks[0], ascending=True)

top_100 = data[(data.adp <= 150) & ((data[ranks[0]] < 100) | (data[ranks[2]] < 100))]

for r in ranks:
    print(r, r2_score(top_100[r], top_100["adp"]))
    
large_diff = top_100.sort_values("abs_diff", ascending=False)[:20].sort_values("diff", ascending=True)

cols = ["name", "pos", "adp"] + ranks + ["diff"]
large_diff[cols][:20].to_csv("top_diff.csv", index=False)

# plt.bar(np.arange(len(large_diff)), large_diff["diff"])
# plt.xticks(np.arange(len(large_diff)), large_diff["name"], rotation=45, ha="right")
# plt.gca().set_title("change in rank with PPR")
# plt.savefig('download.png', bbox_inches='tight')


0.0-rank 0.7233550708237988
0.5-rank 0.6598341842614326
1.0-rank 0.6110835554305178
